In [302]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import redditutils as ru
import word2vecReader as wvr

from gensim.models import Word2Vec
from gensim.models import KeyedVectors

from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.pipeline import make_pipeline

from skopt import BayesSearchCV


%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [289]:
# include below until https://github.com/scikit-optimize/scikit-optimize/issues/718 is resolved
class BayesSearchCV(BayesSearchCV):
    def _run_search(self, x): raise BaseException('Use newer skopt')

In [256]:
df = pd.read_csv('cleaned_shower.csv', index_col = 0)

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [259]:
df['quality'] = df['score'].apply(lambda x: ru.make_labels(x))

In [262]:
df.to_csv('shower_clean.csv')

## Train test split

In [263]:
df = df[df['title'].notna()]

In [265]:
X = df['title']
y = df['quality']

In [276]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y, random_state = 2325)

In [277]:
X_train_short = X_train[:100000]
y_train_short = y_train[:100000]

In [323]:
X_test_short = X_test[:100000]
y_test_short = y_test[:100000]

In [278]:
X_train_shorter = X_train[:5000]
y_train_shorter = y_train[:5000]

In [279]:
X_test_shorter = X_test[:2500]
y_test_shorter = y_test[:2500]

## Word embedding with word2vec

In [270]:
model_path = "./word2vec_twitter_model.bin"
model = KeyedVectors.load_word2vec_format(model_path, binary=True, unicode_errors='ignore')

In [271]:
def make_features(words, model, num_features):
    features = np.zeros(num_features)
    
    model_vocab = set(model.index2word)
    
    num_words = 0
    
    # Loop over words in documents. If the word is in model's vocabulary,
    # generate its feature vector
    for w in words:
        if w in model_vocab:
            num_words += 1
            features = np.add(features, model[w])
            
    # Normalize the feature vector
    features = np.divide(features, num_words)
    
    return features

In [272]:
def document_vecs(docs, model, num_features):
    # Get the average feature vector for each showerthought based on the words it's comprised of
    counter = 0
    
    doc_vector = np.zeros((len(docs), num_features))
    
    for d in docs:
        if counter%100 == 0:
            print(f'Finished document number {counter}')
            
        # Add this document's feature vector to doc_vector
        doc_vector[counter] = make_features(d, model, num_features)
            
        counter += 1
    return doc_vector

In [81]:
features_test = document_vecs(X_test_shorter, model, 400)

Finished document number 0
Finished document number 100
Finished document number 200
Finished document number 300
Finished document number 400
Finished document number 500
Finished document number 600
Finished document number 700
Finished document number 800
Finished document number 900
Finished document number 1000
Finished document number 1100
Finished document number 1200
Finished document number 1300
Finished document number 1400
Finished document number 1500
Finished document number 1600
Finished document number 1700
Finished document number 1800
Finished document number 1900
Finished document number 2000
Finished document number 2100
Finished document number 2200
Finished document number 2300
Finished document number 2400


In [280]:
features_train = document_vecs(X_train_shorter[:1000], model, 400)

Finished document number 0
Finished document number 100
Finished document number 200
Finished document number 300
Finished document number 400
Finished document number 500
Finished document number 600
Finished document number 700
Finished document number 800
Finished document number 900


In [281]:
response_train = y_train[:1000]

[autoreload of redditutils failed: Traceback (most recent call last):
  File "/anaconda3/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/anaconda3/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 368, in superreload
    module = reload(module)
  File "/anaconda3/lib/python3.6/imp.py", line 315, in reload
    return importlib.reload(module)
  File "/anaconda3/lib/python3.6/importlib/__init__.py", line 166, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 618, in _exec
  File "<frozen importlib._bootstrap_external>", line 678, in exec_module
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "/Users/luketibbott/Documents/Metis/project_fletcher/redditutils.py", line 10, in <module>
    from imblearn import SMOTE as sm
ImportError: cannot import name 'SMOTE'
]


In [282]:
def remove_bad_indices(features, response):
    # Remove document-response pairs that failed to be embedded in vector space
    bad_indices = list(np.unique(np.where(np.isnan(features, axis=0)[0])))
    
    features = np.delete(features, bad_indices, axis=0)
    response = np.delete(np.array(response), bad_indices)
    
    return features, response

In [ ]:
one_hunnid_train = document_vecs(X_train_short, model, 400)

Finished document number 0
Finished document number 100
Finished document number 200
Finished document number 300
Finished document number 400
Finished document number 500
Finished document number 600
Finished document number 700
Finished document number 800
Finished document number 900
Finished document number 1000
Finished document number 1100
Finished document number 1200
Finished document number 1300
Finished document number 1400
Finished document number 1500
Finished document number 1600
Finished document number 1700
Finished document number 1800
Finished document number 1900
Finished document number 2000
Finished document number 2100
Finished document number 2200
Finished document number 2300
Finished document number 2400
Finished document number 2500
Finished document number 2600
Finished document number 2700
Finished document number 2800
Finished document number 2900
Finished document number 3000
Finished document number 3100
Finished document number 3200
Finished document numb

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in true_divide
  app.launch_new_instance()


Finished document number 14400
Finished document number 14500
Finished document number 14600
Finished document number 14700
Finished document number 14800
Finished document number 14900
Finished document number 15000
Finished document number 15100
Finished document number 15200
Finished document number 15300
Finished document number 15400
Finished document number 15500
Finished document number 15600
Finished document number 15700
Finished document number 15800
Finished document number 15900
Finished document number 16000
Finished document number 16100
Finished document number 16200
Finished document number 16300
Finished document number 16400
Finished document number 16500
Finished document number 16600
Finished document number 16700
Finished document number 16800
Finished document number 16900
Finished document number 17000
Finished document number 17100
Finished document number 17200
Finished document number 17300
Finished document number 17400
Finished document number 17500
Finished

In [ ]:
pickle.dump(one_hunnid_train, open('100k_train.pkl', 'wb'))

## Throw it in to a Random Forest

Cluster if this doesn't work.

In [125]:
features_train.shape

(5000, 400)

In [301]:
rfc = RandomForestClassifier(n_estimators = 100)

score = cross_val_score(rfc, features, response, cv=3)

print(score)

[0.5764847  0.56722689 0.57297297]


Definitely some improvement with word2vec! ~.56 ROC AUC!

In [287]:
pickle.dump(features, open('1k_features.pkl', 'wb'))
pickle.dump(response, open('1k_response.pkl', 'wb'))

## Clustering

In [300]:
km = KMeans()

rfc = RandomForestClassifier()

pipe = make_pipeline(km, rfc)

tuning_params = {'randomforestclassifier__n_estimators': [50, 100, 150, 200],
                 'kmeans__n_clusters': [i for i in range(1, 20)]}

gs = RandomizedSearchCV(pipe, tuning_params, cv=3, scoring='roc_auc')

gs.fit(features, response)

gs.best_score_

0.5159584307632196

In [319]:
clusters = DBSCAN(eps=0.001, min_samples=10, metric='cosine').fit(features)



clusters.get_params
#rfc = RandomForestClassifier()

#scores = np.mean(cross_val_score(pipe, clusters.labels_, response, cv=3))

#print(scores)

<bound method BaseEstimator.get_params of DBSCAN(algorithm='auto', eps=0.001, leaf_size=30, metric='cosine',
    metric_params=None, min_samples=10, n_jobs=None, p=None)>

In [322]:
clusters.labels_.mean()

-1.0